# Evolution of async in JS

## **The Event Loop - What is it?**
![Event Loop](https://lemycanh.files.wordpress.com/2015/05/eventloop.png)

## **Raw callbacks - the node.js paradigm**
```
fs.readdir(source, function (err, files) {
  if (err) {
    console.log('Error finding files: ' + err)
  } else {
    files.forEach(function (filename, fileIndex) {
      console.log(filename)
      gm(source + filename).size(function (err, values) {
        if (err) {
          console.log('Error identifying file size: ' + err)
        } else {
          console.log(filename + ' : ' + values)
          aspect = (values.width / values.height)
          widths.forEach(function (width, widthIndex) {
            height = Math.round(width / aspect)
            console.log('resizing ' + filename + 'to ' + height + 'x' + height)
            this.resize(width, height).write(dest + 'w' + width + '_' + filename, function(err) {
              if (err) console.log('Error writing file: ' + err)
            })
          }.bind(this))
        }
      })
    })
  }
})
```
**[1]  http://callbackhell.com/**

## **Promises to the rescue**
>_A promise represents the result of an asynchronous operation._

A promise is in one of three different states:
* pending - The initial state of a promise.
* fulfilled - The state of a promise representing a successful operation.
* rejected - The state of a promise representing a failed operation.

Once a promise is fulfilled or rejected, it is immutable (i.e. it can never change again).

**[2]  https://www.promisejs.org/**

### _ex. 1) A succesfully resolved Promise_

In [5]:
var getHamburger = new Promise((resolve, reject) => {
    setTimeout(() => resolve('🍔'), 5000);
});

console.log('Ordering...')
getHamburger.then(result => console.log(result))

Ordering...


[Promise] {}

🍔


### _ex. 2) Rejecting a Promise_

In [ ]:
var getHamburgerMaybe = new Promise((resolve, reject) => {
    setTimeout(() => reject("[OutOfMeatException] Where's the beef?"), 3000);
});

getHamburgerMaybe.catch(error => console.log(error));

### _ex. 3) More advanced Promise examples_

#### _A one-off HTTP request..._

In [2]:
var fetch = require('node-fetch');

fetch('http://localhost:3000/count')
    .then(res => res.json())
    .then(json => console.log(json))
    .catch(error => console.log(error))

[Promise] {}

{"count":11}


#### _Building up a chain of requests and executing them in series..._

In [ ]:
var fetch = require('node-fetch');

/**
 * this function accepts a number, requests a new number, and (eventually) returns their sum
 */
var fetchThenAdd = (previousResult) => {
    return fetch('http://localhost:3000/count')      // fetch a number
        .then(res => {
            console.log('recieved a number!');       // rejoice!
            return res.json();                       // parse the response json
        })
        .then(json => json.count + previousResult);  // add to previous response
};

// sequential fetches
var sequentialRequests = [
    fetchThenAdd,
    fetchThenAdd,
    fetchThenAdd
];

// create the promise chain
var sequence = sequentialRequests.reduce((prev, curr, index) => {
    console.log(`chaining request ${index + 1}`);
    return prev.then(curr);
}, Promise.resolve(0)); // seed it with an initial value

// await the result
sequence
    .then(finalResult => console.log('the sum is', finalResult))
    .catch(error => console.log('the error was', error));

#### _Sending off a batch of requests in parallel..._

In [20]:
var fetch = require('node-fetch');

var fetchNumber = () => {
    return fetch('http://localhost:3000/count')
        .then(res => {
            console.log('recieved a number!');
            return res.json();
        })
        .then(json => json.count)
}

var parallelRequests = [
  fetchNumber,
  fetchNumber,
  fetchNumber
];

// create the batch of promises
var batch = Promise.all(parallelRequests.map(p => p()));

// await the result
batch
    .then(responses => {
        let theSum = responses.reduce((prev, curr) => prev + curr, 0);
        console.log('all done, the responses were', responses);
        console.log('the sum is', theSum);
    })
    .catch(err => console.log(err));

[Promise] {}

recieved a number!
recieved a number!
recieved a number!
all done, the responses were [Array] [1,2,3]
the sum is 6


## Generators - how do they work?

> You can think of generators as processes (pieces of code) that you can pause and resume.

Generators can play three roles:
* Iterators (data producers) - Each yield can return a value via next(), which means that generators can produce sequences of values via loops and recursion. Due to generator objects implementing the interface Iterable, these sequences can be processed by any ES6 construct that supports iterables. Two examples are: for-of loops and the spread operator (...)


* Observers (data consumers): yield can also receive a value from next() (via a parameter). That means that generators become data consumers that pause until a new value is pushed into them via next().


* Coroutines (data producers and consumers)

What generators do not do is give us a way of representing the result of an asynchronous operation. For that, we need a promise.

**[3] http://exploringjs.com/es6/ch_generators.html**

### _ex. 4) A simple generator funciton_

In [2]:
/**
 * this is a generator function definition,
 * it can be paused / resumed while maintaining state internally
 */
var fibonacciGenerator = function* fibonacciGenerator(limit = 10) {
  console.log('initialized fibonacci generator for sequence length', limit);
    
  let i = 0;
  let prev1 = 0;
  let prev2 = 1;
  
  while (i++ < limit) {
   let curr = prev1 + prev2;
   prev1 = prev2;
   prev2 = curr;
      
   yield prev1;
  }
}

// get an interator instance for the generator
var myFib10 = fibonacciGenerator(10);

// move the generator forward manually
console.log(myFib10.next());
console.log(myFib10.next());
console.log(myFib10.next());
console.log(myFib10.next());
console.log(myFib10.next());

initialized fibonacci generator for sequence length 10
{"value":1,"done":false}
{"value":1,"done":false}
{"value":2,"done":false}
{"value":3,"done":false}
{"value":5,"done":false}


In [3]:
// run the remainder of the generator to completion
for (let i of myFib10) {
    console.log(i);
}

8
13
21
34
55


In [4]:
// additional machinery for consuming generators
var myFib20 = fibonacciGenerator(20);
console.log([...myFib20]);

// the iterator is now exhausted
myFib20.next();

initialized fibonacci generator for sequence length 20
[Array] [1,1,2,3,5,8,13,21,34,55,89,144,233,377,610,987,1597,2584,4181,6765]


{"done":true}

## Working together, creating coroutines with Promises and Generators

* Promises give us a way to represent asynchronous operations
* Generators give us a mechanism for (lazily!) operating on streams of data
* Generators can play the role of producer AND consumer

Can we yield a promise, and await it's eventual response?
Yes! ...but not without some external machinery.

**[4] https://www.promisejs.org/generators/**

### _ex. 5) The road to async/await_

In [4]:
// return a wrapper function that automatically moves the state machine forward
var makeAsync = function makeAsync(generator) {
    return () => {
        var iterator = generator.apply(this, arguments);

        // this function consumes the iterator recursively
        function handle(result) {
            // base case: the iterator is empty (complete)
            if (result.done) {
              return Promise.resolve(result.value);
            }

            return Promise.resolve(result.value)
                // attempt to access the generator's .next() value...
                // if no error, progress the generator by providing a value to the preceeding yield
                .then(res => handle(iterator.next(res)))
                
                // in the event of an error, throw the exception back into the generator
                .catch(err => handle(iterator.throw(err)));
        }

        try {
            return handle(iterator.next());
        } catch (ex) {
            return Promise.reject(ex);
        }
    }
}

In [15]:
var fetch = require('node-fetch');

// asynchronous number fetcher
var fetchNumber = () => {
    return fetch('http://localhost:3000/count')
        .then(res => {
            console.log('recieved a number!');
            return res.json();
        })
        .then(json => json.count)
}

// look how synchronous it looks!
var sumOf3 = function* () {
    const number1 = yield fetchNumber();
    const number2 = yield fetchNumber();
    const number3 = yield fetchNumber();
    
    return number1 + number2 + number3;
}

var asyncFn = makeAsync(sumOf3);

// note that async functions still return a promise
asyncFn()
    .then(sum => console.log('the sum is', sum))
    .catch(err => console.log('the error was', err));


[Promise] {}

recieved a number!
recieved a number!
recieved a number!
the sum is 84


## What will it look like in ES7?
Very similar. We'll lose the polyfill (wrapper function), and gain a new syntax:

In [14]:
// this cell won't yet run without Babel :)
async function sumOf3() {
    try {
        const number1 = await fetchNumber();
        const number2 = await fetchNumber();
        const number3 = await fetchNumber();

        return number1 + number2 + number3;
    } catch (err) {
        console.log('the error was', err)
    }
}

sumOf3().then(sum => console.log(sum))

Javascript Error: Unexpected token function

# Thanks for listening!!
![tehm3m3](https://alexperry.io/images/posts/generators/meme2.jpg)